# 0x02_Regression and Bias-Variance Tradeoff

In today's tutorial, we will cover:
- Regression and a specific example, linear regression
- Basis functions and feature engineering
- Bias-variance tradeoff

## 1. Today's dataset

Today we will talk about real estate!

**California housing dataset** is a dataset that contains information about housing prices in California in 1990.
(So, sadly you cannot use it to predict the price of your future house, very likely...)

It includes features such as the number of rooms, the age of the house, and the location of the house.

We would like to predict the median house value.

> 🤔 **THINKING**
>
> - What kind of problem is this, regression or classification? Why?

In [1]:
import pandas as pd

housing_ds = pd.read_csv(
    "data/housing.csv",
)

housing_ds.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


The columns here, quoting the original source:

1. `longitude`: A measure of how far west a house is; a higher value is farther west
2. `latitude`: A measure of how far north a house is; a higher value is farther north
3. `housingMedianAge`: Median age of a house within a block; a lower number is a newer building
4. `totalRooms`: Total number of rooms within a block
5. `totalBedrooms`: Total number of bedrooms within a block
6. `population`: Total number of people residing within a block
7. `households`: Total number of households, a group of people residing within a home unit, for a block
8. `medianIncome`: Median income for households within a block of houses (measured in tens of thousands of US Dollars)
9. `medianHouseValue`: Median house value for households within a block (measured in US Dollars)
10. `oceanProximity`: Location of the house w.r.t ocean/sea

## 2. Vanilla linear regression

Linear regression is described by the following equation:
$$
y = w_0 + w_1 \textbf{x}_1 + w_2 \textbf{x}_2 + ... + w_n \textbf{x}_n = w_0 + \sum_{i=1}^n w_i \textbf{x}_i
$$

where $y$ is the target variable (house price),
$\textbf{x}_i$ are the input vector variables (e.g., number of rooms, age of the house), and $w_i$ are the weights (coefficients) that we need to learn from the data.

The weights are learned by minimizing the mean squared error (MSE) between the predicted and actual values of the target variable.

Let's do this! But, wait a second...

### 2.1 Before you start...

> 🤔 **THINKING**
>
> - Look at the data. Can we directly dump it into a linear regression model? Why or why not?
> - What are the possible problems with the data?

The first problem is that we haven't checked whether we have missing values or not.

It is **VERY COMMON** for datasets to have missing values or buggy source files, and it is a pain, especially if it **breaks your training code that takes a long time** 😭.

Data cleansing is crucial.

In this example, we will remove the NaNs as an example of data cleansing.

In [2]:
housing_ds.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

You can see that there are 207 missing values in the `total_bedrooms` column!

This will break the code.

We can get rid of them by dropping the rows with missing values.

In [3]:
housing_ds = housing_ds.dropna()
housing_ds.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

Voila!

Also, as you can see, `ocean_proximity` is a categorical variable. We cannot use it directly in a linear regression model since $x_i$ must be a number.

Some of you may say, we can make it a number, e.g. `NEAR BAY = 0`, `INLAND = 1`, etc.

In [4]:
# List out the unique values in the 'ocean_proximity' column
housing_ds["ocean_proximity"].unique()

array(['NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'],
      dtype=object)

> 🤔 **THINKING**
>
> - Is this a good idea? Why or why not?
> - Can we compare `NEAR BAY` and `INLAND` with a number? What does it mean to say that `NEAR BAY` is `1` and `INLAND` is `2`?

Let's prove your theory with a set of comparison experiment!

In [5]:
# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
# We control the randomness of the split with a random_state
# This ensures that the split is reproducible
train_ds, test_ds = train_test_split(
    housing_ds,
    test_size=0.2,
    random_state=42,
)

In [6]:
# Create `train_ds_numeric` and `test_ds_numeric` datasets
# In this dataset, the `ocean_proximity` column is mapped to plain numbers
numeric_mapping = {
    "<1H OCEAN": 0,
    "INLAND": 1,
    "ISLAND": 2,
    "NEAR BAY": 3,
    "NEAR OCEAN": 4,
}
train_ds_numeric: pd.DataFrame = train_ds.copy()
train_ds_numeric["ocean_proximity"] = train_ds_numeric["ocean_proximity"].map(
    numeric_mapping
)
test_ds_numeric: pd.DataFrame = test_ds.copy()
test_ds_numeric["ocean_proximity"] = test_ds_numeric["ocean_proximity"].map(
    numeric_mapping
)

In [7]:
train_ds_numeric.head()  # ocean_proximity column is now numerically encoded (0-4)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
17727,-121.80,37.32,14.0,4412.0,924.0,2698.0,891.0,4.7027,227600.0,0
2057,-119.63,36.64,33.0,1036.0,181.0,620.0,174.0,3.4107,110400.0,1
6453,-118.06,34.12,25.0,3891.0,848.0,1848.0,759.0,3.6639,248100.0,1
4619,-118.31,34.07,28.0,2362.0,949.0,2759.0,894.0,2.2364,305600.0,0
15266,-117.27,33.04,27.0,1839.0,392.0,1302.0,404.0,3.5500,214600.0,4


In [8]:
test_ds_numeric.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
14416,-117.24,32.79,20.0,961.0,278.0,525.0,254.0,3.1838,245800.0,4
16383,-121.29,38.01,2.0,6403.0,1116.0,3327.0,957.0,4.4871,137900.0,1
7731,-118.14,33.92,31.0,3731.0,853.0,2313.0,801.0,3.2237,218200.0,0
1410,-122.07,37.94,30.0,1260.0,276.0,707.0,221.0,2.8920,220800.0,3
1335,-121.89,37.99,4.0,2171.0,597.0,928.0,461.0,4.1016,170500.0,1


Voila! Let's test how linear regression works.

As a starting point, let's dump all our columns into a linear regression model and see what happens.

In [9]:
# Regression metrics
# Vanilla linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Define the model
model_numeric = LinearRegression()
# Fit the model
model_numeric.fit(
    train_ds_numeric.drop("median_house_value", axis=1),
    train_ds_numeric["median_house_value"],
)

# Predict the values
predictions_numeric = model_numeric.predict(
    test_ds_numeric.drop("median_house_value", axis=1),
)

# Calculate the mean squared error
mse_numeric = mean_squared_error(
    test_ds_numeric["median_house_value"],
    predictions_numeric,
)
print(f"Mean Squared Error (MSE): {mse_numeric:e}")

Mean Squared Error (MSE): 4.924109e+09


### 2.2 One-hot encoding
We now propose another way to encode categorical variables.

Instead of assigning a number to each category, we can create a new column for each category and assign `1` or `0` to indicate whether the category is present or not.
This is called **one-hot encoding**.

For example, if we have a column `ocean_proximity` with the values `NEAR BAY`, `INLAND`, and `ISLAND`, we can create three new columns:
- `ocean_proximity_NEAR_BAY`: `1` if the value is `NEAR BAY`, `0` otherwise
- `ocean_proximity_INLAND`: `1` if the value is `INLAND`, `0` otherwise
- `ocean_proximity_ISLAND`: `1` if the value is `ISLAND`, `0` otherwise

> 🤔 **THINKING**
>
> - How do you like this encoding? Is it better than the previous one? Why or why not?

In [10]:
train_ds_onehot = pd.get_dummies(
    train_ds,
    columns=["ocean_proximity"],
    drop_first=True,
)
test_ds_onehot = pd.get_dummies(
    test_ds,
    columns=["ocean_proximity"],
    drop_first=True,
)

In [11]:
train_ds_onehot.head()  # ocean_proximity columns are now one-hot encoded

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
17727,-121.80,37.32,14.0,4412.0,924.0,2698.0,891.0,4.7027,227600.0,False,False,False,False
2057,-119.63,36.64,33.0,1036.0,181.0,620.0,174.0,3.4107,110400.0,True,False,False,False
6453,-118.06,34.12,25.0,3891.0,848.0,1848.0,759.0,3.6639,248100.0,True,False,False,False
4619,-118.31,34.07,28.0,2362.0,949.0,2759.0,894.0,2.2364,305600.0,False,False,False,False
15266,-117.27,33.04,27.0,1839.0,392.0,1302.0,404.0,3.5500,214600.0,False,False,False,True


In [12]:
test_ds_onehot.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
14416,-117.24,32.79,20.0,961.0,278.0,525.0,254.0,3.1838,245800.0,False,False,False,True
16383,-121.29,38.01,2.0,6403.0,1116.0,3327.0,957.0,4.4871,137900.0,True,False,False,False
7731,-118.14,33.92,31.0,3731.0,853.0,2313.0,801.0,3.2237,218200.0,False,False,False,False
1410,-122.07,37.94,30.0,1260.0,276.0,707.0,221.0,2.8920,220800.0,False,False,True,False
1335,-121.89,37.99,4.0,2171.0,597.0,928.0,461.0,4.1016,170500.0,True,False,False,False


In [13]:
model_onehot = LinearRegression()
# Fit the model
model_onehot.fit(
    train_ds_onehot.drop("median_house_value", axis=1),
    train_ds_onehot["median_house_value"],
)
# Predict the values
predictions_onehot = model_onehot.predict(
    test_ds_onehot.drop("median_house_value", axis=1),
)
# Calculate the mean squared error
mse_onehot = mean_squared_error(
    test_ds_onehot["median_house_value"],
    predictions_onehot,
)
print(f"Mean Squared Error (MSE): {mse_onehot:e}")

Mean Squared Error (MSE): 4.802174e+09


In [14]:
print(f"Mean square error difference: {mse_onehot - mse_numeric:.2f}")

Mean square error difference: -121935399.31


Hey, not bad! We have, at least in this case, proven that one-hot encoding is better than simply assigning a number to each category.

> 📚 **EXERCISE**
>
> - Dropping NaNs is the simplest way to deal with missing values. However there could be other problems. Can you think of any?
> (One possible hint: Are all of those values reasonable?)
> - Can you think of a solution to the problem you have just identified?
> - Use your new data to train your own model. Is it better than the one we have provided?

In [15]:
# === Your code here ===

## 3. Basis functions and feature engineering

We have seen that linear regression is a powerful tool for regression problems. However, it has its limitations. 

For example, it can only learn linear relationships between the features and the target variable.
This means that if the relationship is non-linear, linear regression will not work well.


**Basis functions** can help you.

A basis function is a function that transforms the input features into a new space where linear regression can be applied.
In terms of formula, we can write it as follows:
$$
y = w_0 + w_1 \phi_1(\textbf{x}) + w_2 \phi_2(\textbf{x}) + ... + w_n \phi_n(\textbf{x}) = w_0 + \textbf{w}^T \phi(\textbf{x})$$

where $\phi(\textbf{x})$ is the vector of basis functions applied to the input feature vector $\textbf{x}$, and $\textbf{w}$ is the vector of weights.

A simple example of a basis function is the polynomial function. 

For example, if we have a single feature $x$, we can create new features by applying the polynomial function:
$$\phi_1(x) = x$$
$$\phi_2(x) = x^2$$
$$\phi_3(x) = x^3$$
$$...$$

This way, we can learn non-linear relationships between the features and the target variable.

How about we try one right now?

We can use `PolynomialFeatures` from `sklearn.preprocessing` to create polynomial features.

In [16]:
from sklearn.preprocessing import PolynomialFeatures

# Create polynomial features
# Here we are using degree=2, which means we will create polynomial features up to the 2rd degree
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly_train = poly.fit_transform(train_ds_onehot.drop("median_house_value", axis=1))
X_poly_test = poly.transform(test_ds_onehot.drop("median_house_value", axis=1))

# We can take a look at what the polynomial features look like
# THIS COULD BE LONG!
# You can use the scrollable output feature in Jupyter Notebook to see the full output
# Use get_feature_names_out to get the names of the features
feature_names = poly.get_feature_names_out(
    input_features=train_ds_onehot.drop("median_house_value", axis=1).columns
)
print(f"Polynomial feature names:\n{feature_names}")

Polynomial feature names:
['longitude' 'latitude' 'housing_median_age' 'total_rooms'
 'total_bedrooms' 'population' 'households' 'median_income'
 'ocean_proximity_INLAND' 'ocean_proximity_ISLAND'
 'ocean_proximity_NEAR BAY' 'ocean_proximity_NEAR OCEAN' 'longitude^2'
 'longitude latitude' 'longitude housing_median_age'
 'longitude total_rooms' 'longitude total_bedrooms' 'longitude population'
 'longitude households' 'longitude median_income'
 'longitude ocean_proximity_INLAND' 'longitude ocean_proximity_ISLAND'
 'longitude ocean_proximity_NEAR BAY'
 'longitude ocean_proximity_NEAR OCEAN' 'latitude^2'
 'latitude housing_median_age' 'latitude total_rooms'
 'latitude total_bedrooms' 'latitude population' 'latitude households'
 'latitude median_income' 'latitude ocean_proximity_INLAND'
 'latitude ocean_proximity_ISLAND' 'latitude ocean_proximity_NEAR BAY'
 'latitude ocean_proximity_NEAR OCEAN' 'housing_median_age^2'
 'housing_median_age total_rooms' 'housing_median_age total_bedrooms'
 'hou

You can see that polynomial feature generates new features by taking the original features and raising them to a power, including combining between them.

As an example, for two features `x1` and `x2`, `PolynomialFeatures` creates new features `x1^2`, `x2^2`, and `x1*x2`.

This rule applies to any number of features.

And when the degree grows, say `degree=3`, we get even more features up to the third degree.
(e.g. `x1^3`, `x2^3`, `x1^2*x2`, `x1*x2^2`, `x1*x2`, `x1^2`, `x2^2`, `x1`, `x2`)

Now, let's see if PolynomialFeatures can help us with our dataset.

In [17]:
model_polynomial_2 = LinearRegression()
# Fit the model
model_polynomial_2.fit(
    X_poly_train,
    train_ds_onehot["median_house_value"],
)
# Predict the values
predictions_polynomial_2 = model_polynomial_2.predict(
    X_poly_test,
)
# Calculate the mean squared error
mse_polynomial_2 = mean_squared_error(
    test_ds_onehot["median_house_value"],
    predictions_polynomial_2,
)
print(f"Mean Squared Error (MSE) using 2-degree polynomial: {mse_polynomial_2:e}")
print(f"Mean square error (MSE) of normal feature: {mse_onehot:e}")
print(
    f"Mean square error difference between 2-deg poly and normal feature: {mse_polynomial_2 - mse_onehot:.2f}"
)

Mean Squared Error (MSE) using 2-degree polynomial: 3.984434e+09
Mean square error (MSE) of normal feature: 4.802174e+09
Mean square error difference between 2-deg poly and normal feature: -817739135.31


Very good! 🎉 Indeed, we can see that using polynomial features can help with the performance of the model.

> 📚 **EXERCISE**
>
> - Try to use `PolynomialFeatures` with a degree of 3 - 5. (If your computer allows, try higher order polynomials. Do note that it becomes time-consuming.) How does the model perform?
> - Observe the pattern from no polynomial features to high degree polynomial features. What do you think is happening? 
> - Hence, is it a good idea to use a high degree polynomial? Why or why not?

In [18]:
# === Your code here ===

> 🤔 **THINKING**
>
> Here is an excerpt from [Wikipedia](https://en.wikipedia.org/wiki/Bias%E2%80%93variance_tradeoff) about the bias-variance tradeoff:
> 
> _"The bias–variance dilemma or bias–variance problem is the conflict in trying to simultaneously minimize these two sources of error that prevent supervised learning algorithms from generalizing beyond their training set:_
>
> - *The bias error is an error from erroneous assumptions in the learning algorithm. High bias can cause an algorithm to miss the relevant relations between features and target outputs (underfitting).*
> - *The variance is an error from sensitivity to small fluctuations in the training set. High variance may result from an algorithm modeling the random noise in the training data (overfitting).*
>
> By the exercise you just did, can you explain how bias-variance tradeoff is demonstrated in the polynomial regression example?